In [1]:
import json
import os
import gzip
from nltk.tokenize import sent_tokenize
import re
import csv
from tqdm.notebook import tqdm

In [2]:
input_fn = '../data/heads/head_val.jsonl.gz'
output_fn = '../out/oxford_comma/head_val_extract.csv'
max_context_len = 1000

In [3]:
data_train = list(gzip.open(input_fn, 'rt'))

In [4]:
fh = open(output_fn, 'wt')
csv_writer = csv.writer(fh)

# write headers
csv_writer.writerow(["line", "sentence", "contains", "index"])

30

In [5]:
regex ="([\w]{4,15}, ){2,}[\w]{4,15}(,)?\s(and|or)\s"

In [6]:
for i, line in tqdm(enumerate(data_train), total=len(data_train)):
    tokenized_line = sent_tokenize(re.sub('\n', ' ', json.loads(line)["text"]))
    
    # maximize the context
    for j in range(len(tokenized_line)-1, -1, -1):
        result = re.search(regex, tokenized_line[j])
        
        if result != None:
            index = result.span()[1]
            
            if result.group(3) == "and":
                index -= 6 
            else:
                index -= 5
                
            #This is so that our index is always at the position that is supposed to be the comma (for non oxford, this is the space)
            has_oxford = result.group(2) == ','
            if not has_oxford:
                index += 1
            
            context = ' '.join(tokenized_line[:j])
            context = context[-max_context_len:]  
            line_with_context = context + ' ' + tokenized_line[j]
            
            csv_writer.writerow([i, line_with_context, has_oxford, len(context) + 1 + index])
        
            # to go to next pile example
            break

  0%|          | 0/20000 [00:00<?, ?it/s]

In [7]:
fh.close()

In [12]:
len(context) + index, line_with_context

(1152,
 '     Health                              Vitek 2             [14](#ccr32374-bib-0014){ref-type="ref"}   Unknown   Unknown   Septicemia      Unknown                             PCR                 [15](#ccr32374-bib-0015){ref-type="ref"}  John Wiley & Sons, Ltd  The studies show that a combination therapy with vancomycin, tetracycline, and the third‐generation cephalosporins is a reliable therapeutic regimen for endocarditis and septicemia infections. [13](#ccr32374-bib-0013){ref-type="ref"} Meanwhile, antibiotics such as vancomycin, the third‐generation cephalosporins, imipenem, meropenem, amikacin are very effective on brain abscesses and CNS infections caused by this bacterium. [5](#ccr32374-bib-0005){ref-type="ref"}, [10](#ccr32374-bib-0010){ref-type="ref"}, [12](#ccr32374-bib-0012){ref-type="ref"}  4. CONCLUSION {#ccr32374-sec-0004} =============  The current study demonstrates the seventh clinical case report about *S\xa0pluranimalium* infection isolated from the neonatal